In [1]:
import os
from tqdm.auto import tqdm

from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import PyPDFDirectoryLoader
from langchain_community.vectorstores import FAISS
from langchain_cohere import ChatCohere, CohereRagRetriever, CohereRerank
from langchain.retrievers import ContextualCompressionRetriever
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough


from oci_cohere_embeddings_utils import OCIGenAIEmbeddingsWithBatch
from utils import format_docs
from my_prompts import prompt_4_answer

from config import EMBED_MODEL, ENDPOINT, COHERE_GENAI_MODEL
from config_private import COMPARTMENT_OCID, COHERE_API_KEY, LANGSMITH_API_KEY

In [2]:
#
# abilitiamo il tracing con LangSmith
#
os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_PROJECT"] = "test_command_r"
os.environ["LANGCHAIN_API_KEY"] = LANGSMITH_API_KEY

# parametri per il retrieval
# Similarity search
TOP_K = 6
# reranking
TOP_N = 4

# la directory che contiene i pdf
DIR_NAME = "./books"
# la directory in cui il vector store è salvato
FAISS_DIR = "./faiss_index"

# parametri LLM Cohere
TEMPERATURE = 0.0
MAX_TOKENS = 1024

In [3]:
embed_model = OCIGenAIEmbeddingsWithBatch(
    auth_type="API_KEY",
    model_id=EMBED_MODEL,
    service_endpoint=ENDPOINT,
    compartment_id=COMPARTMENT_OCID,
)

cohere_rerank = CohereRerank(cohere_api_key=COHERE_API_KEY, top_n=TOP_N)

# prompt per la genereazione della risposta finale


#
# LLM
#
llm = ChatCohere(
    cohere_api_key=COHERE_API_KEY,
    model=COHERE_GENAI_MODEL,
    max_tokens=MAX_TOKENS,
    temperature=TEMPERATURE,
)

In [4]:
#
# qui vettorizza tutti i chunks oppure
# li legge se salvati in locale
#
if os.path.exists(FAISS_DIR):
    print("Loading Vector Store from local dir...")

    v_store = FAISS.load_local(
        FAISS_DIR, embed_model, allow_dangerous_deserialization=True
    )
else:
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=3000,
        chunk_overlap=50,
        length_function=len,
        is_separator_regex=False,
    )

    loader = PyPDFDirectoryLoader(path=DIR_NAME, glob="*.pdf")

    # document are pages
    docs = loader.load_and_split(text_splitter=text_splitter)

    print(f"Loaded {len(docs)} chunks...")

    print(f"Indexing books...")
    v_store = FAISS.from_documents(docs, embed_model)

    v_store.save_local(FAISS_DIR)

Loaded 2790 chunks...
Indexing books...


  0%|          | 0/31 [00:00<?, ?it/s]

In [5]:
base_retriever = v_store.as_retriever()

#
# add a reranker
#
compression_retriever = ContextualCompressionRetriever(
    base_compressor=cohere_rerank, base_retriever=base_retriever
)

In [6]:
#
# create the entire chain
#
rag_chain = (
    {"context": compression_retriever | format_docs, "question": RunnablePassthrough()}
    | prompt_4_answer
    | llm
    | StrOutputParser()
)

In [7]:
def answer(rag_chain, question):
    answer = rag_chain.invoke(question)

    print(f"Question: {question}")
    print("")
    print(answer)
    print("")

In [8]:
question = """Puoi scrivere una mail, per un cliente di nome Luigi Saetta, 
           in cui spieghi quali vantaggi Oracle Data Guard offre per il Disaster Recovery?"""

In [9]:
%%time
answer(rag_chain, question)

  0%|          | 0/1 [00:00<?, ?it/s]

Question: Puoi scrivere una mail, per un cliente di nome Luigi Saetta, 
           in cui spieghi quali vantaggi Oracle Data Guard offre per il Disaster Recovery?

Caro Luigi Saetta,

Voglio portarle all'attenzione i vantaggi di Oracle Data Guard per le soluzioni di disaster recovery. Oracle Data Guard è una funzionalità potente e completa che aiuta a garantire l'elevata disponibilità, la protezione dei dati e il recupero in caso di disastri.

I vantaggi chiave di utilizzare Oracle Data Guard per il disaster recovery includono:

1. Elevata disponibilità: Oracle Data Guard consente di creare un numero massimo di 30 repliche stand-by dei database primari. Queste repliche stand-by possono essere utilizzate per garantire un'elevata disponibilità dei dati e ridurre al minimo i tempi di inattività in caso di guasti.

2. Protezione dei dati: Oracle Data Guard fornisce un'ampia serie di servizi per la creazione, la gestione e il monitoraggio di database stand-by. Ciò aiuta a proteggere i dati 

In [10]:
question = """Puoi elencare i sintomi del long covid negli adulti?"""

answer(rag_chain, question)

  0%|          | 0/1 [00:00<?, ?it/s]

Question: Puoi elencare i sintomi del long covid negli adulti?

I sintomi del long COVID negli adulti possono variare ampiamente da persona a persona e possono persistere o ricorrere per mesi dopo l'infezione iniziale da SARS-CoV-2. I sintomi più comuni includono:

1. Affaticamento
2. Dispnea (difficoltà a respirare)
3. Dolori muscolari e articolari
4. Mal di testa
5. Perdita dell'olfatto e del gusto
6. Dolore al petto
7. Insomnia
8. Nausea e vomito
9. Diarrea
10. Febbre bassa

Inoltre, i pazienti possono sperimentare sintomi neurologici come nebbia mentale, difficoltà di concentrazione e dimenticanza, nonché sintomi psicologici come ansia e depressione. Il long COVID può anche causare dolori alle articolazioni, mal di gola, dolori addominali e perdita di peso.

In alcuni casi, i sintomi del long COVID possono essere gravi e compromettere la capacità di una persona di svolgere le normali attività quotidiane. È importante notare che il long COVID può verificarsi sia in pazienti che hann